In [12]:
import urllib3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  
http = urllib3.PoolManager()

In [6]:
r = http.request('GET', 'http://s.cafef.vn/Ajax/Events_RelatedNews_New.aspx?symbol=A32&floorID=0&configID=0&PageIndex=1&PageSize=10000&Type=2')

In [14]:
soup = BeautifulSoup(r.data, "html.parser")

In [15]:
data = soup.find("ul", {"class": "News_Title_Link"})

In [18]:
raw = data.find_all('li')

In [48]:
data_dicts = []
for row in raw:
    row_dict = {}
    row_dict['newsdate'] = row.span.text
    row_dict['title'] = row.a.text
    row_dict['url']  = "http://s.cafef.vn/" + str(row.a['href'])
    data_dicts.append(row_dict)
    
data_dicts

    

[{'newsdate': '10/07/2019 09:34',
  'title': 'A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đã mua 2.800 CP',
  'url': 'http://s.cafef.vn//a32-314278/a32-ong-nguyen-the-anh-pho-chu-tich-hdqt-da-mua-2800-cp.chn'},
 {'newsdate': '01/07/2019 14:21',
  'title': 'A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đăng ký mua 2.800 CP',
  'url': 'http://s.cafef.vn//a32-312903/a32-ong-nguyen-the-anh-pho-chu-tich-hdqt-dang-ky-mua-2800-cp.chn'},
 {'newsdate': '03/06/2019 08:55',
  'title': 'Điểm danh những doanh nghiệp chốt quyền nhận cổ tức bằng tiền, bằng cổ phiếu và cổ phiếu thưởng tuần này',
  'url': 'http://s.cafef.vn//a32-309371/diem-danh-nhung-doanh-nghiep-chot-quyen-nhan-co-tuc-bang-tien-bang-co-phieu-va-co-phieu-thuong-tuan-nay.chn'},
 {'newsdate': '28/05/2019 13:46',
  'title': 'Lịch chốt quyền nhận cổ tức bằng tiền của 8 doanh nghiệp',
  'url': 'http://s.cafef.vn//a32-308683/lich-chot-quyen-nhan-co-tuc-bang-tien-cua-8-doanh-nghiep.chn'},
 {'newsdate': '28/05/2019 13:27',
  'title': 'A32: 6.6.

In [49]:
pd.DataFrame(data_dicts)

,newsdate,title,url
0,10/07/2019 09:34,A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đã...,http://s.cafef.vn//a32-314278/a32-ong-nguyen-t...
1,01/07/2019 14:21,A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đă...,http://s.cafef.vn//a32-312903/a32-ong-nguyen-t...
2,03/06/2019 08:55,Điểm danh những doanh nghiệp chốt quyền nhận c...,http://s.cafef.vn//a32-309371/diem-danh-nhung-...
3,28/05/2019 13:46,Lịch chốt quyền nhận cổ tức bằng tiền của 8 do...,http://s.cafef.vn//a32-308683/lich-chot-quyen-...
4,28/05/2019 13:27,"A32: 6.6.2019, ngày GDKHQ trả cổ tức bằng tiền...",http://s.cafef.vn//a32-308769/a32-662019-ngay-...
5,22/05/2019 09:49,A32: Thông báo về việc chi cổ tức đợt 2 năm 2018,http://s.cafef.vn//a32-307974/a32-thong-bao-ve...
6,07/05/2019 16:44,A32: Nghị quyết Đại hội đồng cổ đông thường ni...,http://s.cafef.vn//a32-306053/a32-nghi-quyet-d...
7,19/04/2019 14:25,A32: Tài liệu họp Đại hội đồng cổ đông,http://s.cafef.vn//a32-302704/a32-tai-lieu-hop...
8,17/04/2019 16:31,A32: Tài liệu họp Đại hội đồng cổ đông,http://s.cafef.vn//a32-302221/a32-tai-lieu-hop...
9,05/04/2019 14:29,"A32: 16.4.2019, ngày GDKHQ Tham dự Đại hội đồn...",http://s.cafef.vn//a32-300001/a32-1642019-ngay...
